In [ ]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
#import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import shutil

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
import logging
from skimage.io import imread
#from torchviz import make_dot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from torch.autograd import Variable
from torch.optim import *
from scipy.io import loadmat, savemat
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, ConcatDataset, DataLoader, SubsetRandomSampler
#import torchio as tio
from torch.utils.tensorboard import SummaryWriter

import matplotlib.cm as cm
import matplotlib.animation as animation
#import plotly.graph_objects as go
#import plotly.io as pio
#pio.renderers.default = "notebook"
import math

from utils.dataset_utils import *
from utils.classifier_utils import *
from utils.neural_networks import *

logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt="%Y-%m-%d %H:%M:%S")

In [ ]:
info = {}
info["ext"] = "mat"
#info["ext"] = "nii"
dir_path = "../RSNA-BTC-Datasets/train_"+info["ext"]
test_dir_path = "../RSNA-BTC-Datasets/test_"+info["ext"]
tumor_only_dir_path = "../RSNA-BTC-Datasets/ec_train_"+info["ext"]
tumor_only_test_dir_path = "../RSNA-BTC-Datasets/ec_test_"+info["ext"]
no_tumor_dir_path = "../RSNA-BTC-Datasets/no_tumor_train_"+info["ext"]
ext_test_1_dir_path = "../RSNA-BTC-Datasets/brats18_"+info["ext"]
ext_test_0_dir_path = "../RSNA-BTC-Datasets/OpenNeuroDS000221_ss_"+info["ext"]
new_dir_path = "../RSNA-BTC-Datasets/UPENN-GBM_"+info["ext"]

In [ ]:
"""
import os
# assign directory
full_0 = os.listdir(dir_path+"/KLF/0")
filtered_0 = os.listdir(tumor_only_dir_path+"/KLF/0")
full_0.sort()
filtered_0.sort()
print(full_0)
print(filtered_0)
negative_0 = list(set(full_0)-set(filtered_0))
negative_0.sort()
print(negative_0)


# iterate over files in
# that directory
for filename in os.listdir(dir_path+"/KLF/0"):
    if filename in negative_0:
        original_path = os.path.join(dir_path+"/KLF/0", filename)
        dest_path = os.path.join("../RSNA-BTC-Datasets/no_tumor_train_"+info["ext"]+"/KLF/0",filename)
        shutil.copy(original_path, dest_path)
"""

In [ ]:
chosen_net  = "sim" #3du, sim, alt, tun
if chosen_net == "alt":
    info["dims"] = 2 #3,2
    info["batch_size"] = 1#8,4,1
    sel_slices = 1 #None,1
elif chosen_net == "tun":
    info["dims"] = 3 #3,2
    info["batch_size"] = 1#8,4,1
    sel_slices = None #None,1
else:
    info["dims"] = 3 #3,2
    info["batch_size"] = 4#8,4,1
    sel_slices = None #None,1
info["mri_types"] = ["KLF"] #["KLF"] ["FLAIR"] ["T1w"] ["FLAIR", "T2w"]
info["is_depth_wise"] = True
info["is_transformed"] = False
transform = None
info["output_size"] = 1
info["is_multistep"] = True
info["lr"] = 0.0001
info["is_adam"] = True
info["momentum"] = 0.9
info["is_10_fold"] = False
info["val_total_ratio"] = 0.2
info["is_internal_dataset"] = True
# Train and Val Origin:
# - m: Competition training or validation set
# - M: Competition training and validation set
# - f: Competition (tumor only) training or validation set
# - F: Competition (tumor only) training and validation set
# - t: External training or validation set
# - n: UPENN (tumor only) training or validation set
# - N: UPENN (tumor only) training and validation set
# - fn: Competition (tumor only) + UPENN (tumor only) training or validation
# - FN: Competition (tumor only) + UPENN (tumor only) training and validation
info["train_origin"] = "mn"
info["val_origin"] = "mn"
size = len(info["mri_types"])

"""
chosen_net  = "alt" #3du
info["dims"] = 2
info["mri_types"] = ["KLF"]
info["batch_size"] = 1#8
info["is_depth_wise"] = True
info["is_transformed"] = False
transform = None
info["output_size"] = 1
info["is_multistep"] = True
info["lr"] = 0.0005
info["is_adam"] = True
info["momentum"] = 0.9
info["is_10_fold"] = False
info["val_total_ratio"] = 0.2
info["is_internal_dataset"] = True
size = len(info["mri_types"])
sel_slices = 1
"""

"""
chosen_net  = "eff" #3du
info["dims"] = 3
info["mri_types"] = ["FLAIR"]
info["batch_size"] = 4#8
info["is_depth_wise"] = False
info["is_transformed"] = False
transform = None
info["output_size"] = 1
info["is_multistep"] = True
info["lr"] = 0.0005
info["is_adam"] = True
info["momentum"] = 0.9
info["is_10_fold"] = False
info["val_total_ratio"] = 0.1
info["is_internal_dataset"] = True
size = len(info["mri_types"])
sel_slices = None
"""
print(info)

In [ ]:
import importlib
from utils import dataset_utils
importlib.reload(dataset_utils)

packs = dataset_utils.generate_datasets(['KLF','FLAIR','T1w'], info, transform, sel_slices, 
                          m_path=dir_path, n_path=new_dir_path, f_path=tumor_only_dir_path, h_path=no_tumor_dir_path)
print(packs)

In [ ]:
import importlib
from utils import dataset_utils
importlib.reload(dataset_utils)

loader_packs = dataset_utils.get_loaders(packs, info)
print(loader_packs)

In [ ]:
# Model check
model = try_and_get_model(chosen_net, info["dims"], size, info["is_depth_wise"], info["output_size"])
print(count_parameters(model))
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info("Using CUDA...")

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
#print(loader_packs['KLF']['m_dataloaders'])
selected_type = info["mri_types"][0]
print(selected_type)
m_dataloader = loader_packs[selected_type]['m_dataloaders']#m_dataloaders[0]
# Competition Train
m_train_loader = m_dataloader[0]
# Competition Val
m_val_loader = m_dataloader[1]
# Competition Train + Val
m_train_val_loader = m_dataloader[3]#m_loader
# Competition Test
m_test_loader = m_dataloader[2]

#t_dataloader = loader_packs[selected_type]['t_dataloaders']#t_dataloaders[0]
# External Train
#t_train_loader = t_dataloader[0]
# External Val
#t_val_loader = t_dataloader[1]
# External Test
#t_test_loader = t_dataloader[2]

n_dataloader = loader_packs[selected_type]['n_dataloaders']#n_dataloaders[0]
# UPENN Train
n_train_loader = n_dataloader[0]
# UPENN Val
n_val_loader = n_dataloader[1]
# UPENN Train + Val
n_train_val_loader = n_dataloader[3]#n_loader
# UPENN Test
n_test_loader = n_dataloader[2]

if selected_type == "KLF":
    f_dataloader = loader_packs[selected_type]['f_dataloaders']#f_dataloaders[0]
    # Competition (Tumor Only) Train
    f_train_loader = f_dataloader[0]
    # Competition (Tumor Only) Val
    f_val_loader = f_dataloader[1]
    # Competition (Tumor Only) Train + Val
    f_train_val_loader = f_dataloader[3]
    # Competition (Tumor Only) Test
    f_test_loader = f_dataloader[2]
    
    h_dataloader = loader_packs[selected_type]['h_dataloaders']#h_dataloaders[0]
    # Competition (No Tumor) Train
    h_train_loader = h_dataloader[0]
    # Competition (No Tumor) Val
    h_val_loader = h_dataloader[1]
    # Competition (No Tumor) Train + Val
    h_train_val_loader = h_dataloader[3]
    # Competition (No Tumor) Test
    h_test_loader = h_dataloader[2]
    
    fn_dataloader = loader_packs[selected_type]['fn_dataloaders']#fn_dataloaders[0]
    # Competition (Tumor Only) + UPENN Train
    fn_train_loader = fn_dataloader[0]
    # Competition (Tumor Only) + UPENN Val
    fn_val_loader = fn_dataloader[1]
    # Competition (Tumor Only) + UPENN Train + Val
    fn_train_val_loader = fn_dataloader[3]
    # Competition (Tumor Only) + UPENN Test
    fn_test_loader = fn_dataloader[2]

In [ ]:
m_model = try_and_get_model(chosen_net, info["dims"], size, info["is_depth_wise"], info["output_size"])

In [ ]:
# Methylation Detector
train_loader = m_train_loader
to = info["train_origin"]
if to == "m":
    train_loader = m_train_loader
elif to == "M":
    train_loader = m_train_val_loader
elif to == "f":
    train_loader = f_train_loader
elif to == "F":
    train_loader = f_train_val_loader
elif to == "h":
    train_loader = h_train_loader
elif to == "n":
    train_loader = n_train_loader
elif to == "N":
    train_loader = n_train_val_loader
elif to == "t":
    train_loader = t_train_loader
elif to == "fn":
    train_loader = fn_train_loader
elif to == "FN":
    train_loader == fn_train_val_loader
    
val_loader = m_val_loader
vo = info["val_origin"]
if vo == "m":
    val_loader = m_val_loader
elif vo == "M":
    val_loader = m_train_val_loader
elif vo == "f":
    val_loader = f_val_loader
elif vo == "F":
    val_loader = f_train_val_loader
elif vo == "h":
    val_loader = h_val_loader
elif vo == "n":
    val_loader = n_val_loader
elif vo == "N":
    val_loader = n_train_val_loader
elif vo == "t":
    val_loader = t_val_loader
elif vo == "fn":
    val_loader = fn_val_loader
elif vo == "FN":
    val_loader == fn_train_val_loader
    
print(to, vo)

In [ ]:
print(info)
m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
                       train_loader=train_loader, valid_loader=val_loader)
##m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
##                       train_loader=m_train_loader, valid_loader=m_val_loader)

#m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
#                       train_loader=n_train_loader, valid_loader=n_val_loader)

In [ ]:
print(m_modelfile)

In [ ]:
"""
folders_list = [
    "RSNAClassifierSingle3D-DW-SO-KLF_Jan27_19-25-03",
    "RSNAAlternativeClassifierSingle2D-DW-SO-KLF_Jul07_17-25-17",
    "RSNAClassifierSingle3D-DW-SO_Jan27_10-46-07",
]
"""

folders_list = [
    #"RSNAClassifierSingle3D-DW-SO-KLF_Jul10_21-39-50", #m
    "RSNAClassifierSingle3D-DW-SO-KLF_Aug05_22-07-19", #m
    "RSNAClassifierSingle3D-DW-SO_Aug03_18-40-51", #m flair
    "RSNAClassifierSingle3D-DW-SO_Aug03_20-14-30", #m t1w
    "RSNAClassifierSingle3D-DW-SO-KLF_Aug05_17-47-49", #n
    "RSNAClassifierSingle3D-DW-SO-KLF_Aug05_18-26-58", #f
    "RSNAClassifierSingle3D-DW-SO-KLF_Aug02_18-42-47", #fn
    "RSNAAlternativeClassifierSingle2D-DW-SO-KLF_Aug02_11-12-02", #m
    "RSNAAlternativeClassifierSingle2D-DW-SO-KLF_Aug05_20-21-59", #n
    "RSNAAlternativeClassifierSingle2D-DW-SO-KLF_Aug02_12-17-49", #f
    "RSNAAlternativeClassifierSingle2D-DW-SO-KLF_Aug02_12-58-14" #fn 
]
"""
choice = 0
model_common_path = "../RSNA-BTC-Datasets/out_models/"
model_folder = model_common_path + folders_list[choice]

model_basefile, info = get_best_model(model_folder)
m_modelfile = model_folder + "/" + model_basefile
transform = None
size = len(info["mri_types"])
sel_slices = 1 if choice == 1 else None
chosen_net = "alt" if choice == 1 else "sim"

#m_modelfile = m_modelfile_3d
print(m_modelfile)
print(info)
"""
print(folders_list)

In [ ]:
rc = pd.read_csv("RisultatoClassificazione.csv", dtype={"ID": object})
print(rc)
fixed_rc = rc.loc[rc["TUMORE"] == 0]
h_ids = fixed_rc["ID"]
h_preds = fixed_rc["PRED"]
h_probs = fixed_rc["PROB_1"]#[item["PROB_0"] for item in fixed_rc if item["PRED"] == 0 else item["PROB_1"]]

test_rc = fixed_rc.sample(frac=0.2)
h_ids = list(test_rc["ID"])
h_ids = [idx.zfill(5) for idx in h_ids]
h_y = list(np.zeros(len(h_ids)))
h_preds = list(test_rc["PRED"])
h_probs = list(test_rc["PROB_1"])#[item["PROB_0"] for item in fixed_rc if item["PRED"] == 0 else item["PROB_1"]]

print(h_ids)
print(h_y)
print(h_preds)
print(h_probs)

In [ ]:
metrics = pd.DataFrame({"model": [], "AUC": [], "acc": [], "spec": [], "sens": [], "prec": []})
for folder in folders_list:
    model_common_path = "../RSNA-BTC-Datasets/out_models/"
    model_folder = model_common_path + folder

    model_basefile, info = get_best_model(model_folder)
    m_modelfile = model_folder + "/" + model_basefile
    transform = None
    size = len(info["mri_types"])
    sel_slices = 1 if info["dims"] == 2 else None
    chosen_net = "alt" if info["dims"] == 2 else "sim"

    #m_modelfile = m_modelfile_3d
    print(m_modelfile)
    print(info)
    
    packs = generate_datasets(['KLF','FLAIR','T1w'])
    print(packs)
    
    loader_packs = get_loaders(packs)
    print(loader_packs)
    
    # Model check
    model = try_and_get_model(chosen_net, info["dims"], size, info["is_depth_wise"], info["output_size"])
    print(count_parameters(model))
    print(model)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info("Using CUDA...")
    
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print(loader_packs['KLF']['m_dataloaders'])
    selected_type = info["mri_types"][0]
    print(selected_type)
    m_dataloader = loader_packs[selected_type]['m_dataloaders']#m_dataloaders[0]
    # Competition Train
    m_train_loader = m_dataloader[0]
    # Competition Val
    m_val_loader = m_dataloader[1]
    # Competition Train + Val
    m_train_val_loader = m_dataloader[3]#m_loader
    # Competition Test
    m_test_loader = m_dataloader[2]

    t_dataloader = loader_packs[selected_type]['t_dataloaders']#t_dataloaders[0]
    # External Train
    t_train_loader = t_dataloader[0]
    # External Val
    t_val_loader = t_dataloader[1]
    # External Test
    t_test_loader = t_dataloader[2]

    n_dataloader = loader_packs[selected_type]['n_dataloaders']#n_dataloaders[0]
    # UPENN Train
    n_train_loader = n_dataloader[0]
    # UPENN Val
    n_val_loader = n_dataloader[1]
    # UPENN Train + Val
    n_train_val_loader = n_dataloader[3]#n_loader
    # UPENN Test
    n_test_loader = n_dataloader[2]

    if selected_type == "KLF":
        f_dataloader = loader_packs[selected_type]['f_dataloaders']#f_dataloaders[0]
        # Competition (Tumor Only) Train
        f_train_loader = f_dataloader[0]
        # Competition (Tumor Only) Val
        f_val_loader = f_dataloader[1]
        # Competition (Tumor Only) Train + Val
        f_train_val_loader = f_dataloader[3]
        # Competition (Tumor Only) Test
        f_test_loader = f_dataloader[2]

        h_dataloader = loader_packs[selected_type]['h_dataloaders']#h_dataloaders[0]
        # Competition (No Tumor) Train
        h_train_loader = h_dataloader[0]
        # Competition (No Tumor) Val
        h_val_loader = h_dataloader[1]
        # Competition (No Tumor) Train + Val
        h_train_val_loader = h_dataloader[3]
        # Competition (No Tumor) Test
        h_test_loader = h_dataloader[2]

        fn_dataloader = loader_packs[selected_type]['fn_dataloaders']#fn_dataloaders[0]
        # Competition (Tumor Only) + UPENN Train
        fn_train_loader = fn_dataloader[0]
        # Competition (Tumor Only) + UPENN Val
        fn_val_loader = fn_dataloader[1]
        # Competition (Tumor Only) + UPENN Train + Val
        fn_train_val_loader = fn_dataloader[3]
        # Competition (Tumor Only) + UPENN Test
        fn_test_loader = fn_dataloader[2]
        
    m_model = try_and_get_model(chosen_net, info["dims"], size, info["is_depth_wise"], info["output_size"])
    
    # Methylation Detector
    train_loader = m_train_loader
    to = info["train_origin"]
    if to == "m":
        train_loader = m_train_loader
    elif to == "M":
        train_loader = m_train_val_loader
    elif to == "f":
        train_loader = f_train_loader
    elif to == "F":
        train_loader = f_train_val_loader
    elif to == "h":
        train_loader = h_train_loader
    elif to == "n":
        train_loader = n_train_loader
    elif to == "N":
        train_loader = n_train_val_loader
    elif to == "t":
        train_loader = t_train_loader
    elif to == "fn":
        train_loader = fn_train_loader
    elif to == "FN":
        train_loader == fn_train_val_loader

    val_loader = m_val_loader
    vo = info["val_origin"]
    if vo == "m":
        val_loader = m_val_loader
    elif vo == "M":
        val_loader = m_train_val_loader
    elif vo == "f":
        val_loader = f_val_loader
    elif vo == "F":
        val_loader = f_train_val_loader
    elif vo == "h":
        val_loader = h_val_loader
    elif vo == "n":
        val_loader = n_val_loader
    elif vo == "N":
        val_loader = n_train_val_loader
    elif vo == "t":
        val_loader = t_val_loader
    elif vo == "fn":
        val_loader = fn_val_loader
    elif vo == "FN":
        val_loader == fn_train_val_loader

    print(to, vo)
    
    print("\nPREDICTIONS:")
    print(folder)
    print(to, vo)
    print("Test set: m (A)")
    if info["train_origin"] == "f":
        test_loader = f_test_loader
        f_test_pred, ids, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, test_loader, size)
        ids = list(ids) + h_ids
        y = list(y) + h_y
        y_pred = list(y_pred) + h_preds
        y_prob = list(y_prob) + h_probs
        print("Fixed metrics:")
        pd_metrics = get_metrics(y, y_pred, y_prob, f"{folder}_{to}_A")
        
    else:
        test_loader = m_test_loader
        m_test_pred, ids, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, test_loader, size)
        pd_metrics = get_metrics(y, y_pred, y_prob, f"{folder}_{to}_A")
        
    preddfA = pd.DataFrame({"BraTS21ID": ids, "MGMT_real_value": y, "MGMT_pred_value": y_pred, "MGMT_prob_value": y_prob}) 
    preddfA = preddfA.sort_values(by="BraTS21ID")
    preddfA.to_csv(f"pred_metrics/{folder}_{to}_A.csv", index=False)
    metrics = metrics.append(pd_metrics, ignore_index=True)
    #pd_metrics.to_csv(f"pred_metrics/{folder}_{to}_A_metrics.csv", index=False)
    
    print("Test set: n (B)")
    test_loader = n_test_loader
    n_test_pred, ids, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, test_loader, size)
    pd_metrics = get_metrics(y, y_pred, y_prob, f"{folder}_{to}_B")
    preddfB = pd.DataFrame({"BraTS21ID": ids, "MGMT_real_value": y, "MGMT_pred_value": y_pred, "MGMT_prob_value": y_prob}) 
    preddfB = preddfB.sort_values(by="BraTS21ID")
    preddfB.to_csv(f"pred_metrics/{folder}_{to}_B.csv", index=False)
    metrics = metrics.append(pd_metrics, ignore_index=True)
    
    print("----------\n")
    
metrics.to_csv(f"pred_metrics/all_metrics.csv", index=False)

In [ ]:
print(info)
m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
                       train_loader=train_loader, valid_loader=val_loader)
##m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
##                       train_loader=m_train_loader, valid_loader=m_val_loader)

#m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
#                       train_loader=n_train_loader, valid_loader=n_val_loader)

In [ ]:
print(info)
m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
                       train_loader=train_loader, valid_loader=val_loader)
##m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
##                       train_loader=m_train_loader, valid_loader=m_val_loader)

#m_modelfile = train_mri_type(m_model, device, info, epochs=150, pat=30, 
#                       train_loader=n_train_loader, valid_loader=n_val_loader)

In [ ]:
print(m_modelfile)

In [ ]:
# C+N
#cn_test_pred, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, n_test_loader, size)

In [ ]:
print(cc_test_pred)

In [ ]:
# N+C
nc_test_pred, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, m_test_loader, size)

In [ ]:
from sklearn.metrics import roc_auc_score
def plot_confusion_matrix(y_valid, y_pred,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = confusion_matrix(y_valid, y_pred)
    classes = ["0", "1"]

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.clim(0,cm.max())
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = (cm.max() - cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
#m_old_modelfile = "../RSNA-BTC-Datasets/out_models/RSNAClassifierSingle3D-DW-SO-KLF_Jul07_16-02-56/RSNAClassifierSingle3D-DW-SO-KLF-e59-loss2.968-acc0.617-auc0.553.pth"
#m_old_modelfile = "../RSNA-BTC-Datasets/out_models/RSNAAlternativeClassifierSingle2D-DW-SO-KLF_Jul07_17-25-17/RSNAAlternativeClassifierSingle2D-DW-SO-KLF-e137-loss2.965-acc0.675-auc0.667.pth"
#m_old_modelfile = "../RSNA-BTC-Datasets/out_models/RSNAClassifierSingle3D-DW-SO_Jan27_10-46-07/RSNAClassifierSingle3D-DW-SO-e13-loss0.012-acc1.000-auc1.000.pth"
#m_old_modelfile = "../RSNA-BTC-Datasets/out_models/RSNAClassifierSingle3D-DW-SO-KLF_Jan27_19-25-03/RSNAClassifierSingle3D-DW-SO-KLF-e65-loss0.004-acc1.000-auc1.000.pth"
#m_modelfile = "../RSNA-BTC-Datasets/out_models/RSNAClassifierSingle3D-DW-SO-KLF_Jul10_20-59-21/RSNAClassifierSingle3D-DW-SO-KLF-e4-loss1.238-acc0.550-auc0.591.pth"
# C+N
#cn_test_pred, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, n_test_loader, size)
#cn_test_pred_f, X_f, y_f, y_pred_f, y_prob_f = predict(m_model, device, m_modelfile, loader_packs['FLAIR']['n_dataloaders'][2], size)
# C+C
test_loader = fn_test_loader
cc_test_pred, ids, X, y, y_pred, y_prob = classifier_utils.predict(m_model, device, m_modelfile, test_loader, size)

In [ ]:
test_loader = n_test_loader
cc_test_pred, ids, X, y, y_pred, y_prob = predict(m_model, device, m_modelfile, test_loader, size)

In [ ]:
import importlib
importlib.reload(classifier_utils)

test_loader = m_test_loader
cc_test_pred, ids, X, y, y_pred, y_prob = classifier_utils.predict(m_model, device, m_modelfile, test_loader, size)

In [ ]:
#test_loader = h_test_loader
#cc_test_pred, ids, X, y, y_pred, y_prob = classifier_utils.predict(m_model, device, m_modelfile, test_loader, size)

In [ ]:
#plot_confusion_matrix(y, y_pred)
auc = roc_auc_score(y, y_prob)
print(auc)
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y, y_prob)

# Calculate the G-mean
gmean = np.sqrt(tpr * (1 - fpr))

# Find the optimal threshold
index = np.argmax(gmean)
thresholdOpt = round(thresholds[index], ndigits = 4)
gmeanOpt = round(gmean[index], ndigits = 4)
fprOpt = round(fpr[index], ndigits = 4)
tprOpt = round(tpr[index], ndigits = 4)
print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k-')
plt.plot(fpr, tpr, label='CNN(area = {:.3f})'.format(auc))
plt.plot(fprOpt, tprOpt, 'ro')
plt.text(fprOpt+0.05,tprOpt-0.05,thresholdOpt)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
plt.savefig("/home/salvatorecapuozzo/Scrivania/ROC_curve_m.png")

In [ ]:
for i in range(1,10):
    j = 0.1*i
    y_pred_new = (np.array(y_prob) > j).astype(int)
    print(f"{j:.1f}")
    #print(y_pred)
    acc = [1 if yy == out else 0 for (yy,out) in zip(y,y_pred_new)].count(1)/len(y_pred_new)
    total_0_count = y.count(0)
    total_1_count = y.count(1)
    total_1_pred_count = list(y_pred_new).count(1)
    true_0 = [1 if yy == out and yy == 0 else 0 for (yy,out) in zip(y,y_pred_new)].count(1)
    true_1 = [1 if yy == out and yy == 1 else 0 for (yy,out) in zip(y,y_pred_new)].count(1)
    spec = true_0/total_0_count
    sens = true_1/total_1_count
    if total_1_pred_count != 0:
        prec = true_1/total_1_pred_count
    else:
        prec = 0
    print(f"Prediction Accuracy: {acc:.4f}")
    print(f"Prediction Specificity: {spec:.4f}")
    print(f"Prediction Sensitivity: {sens:.4f}")
    print(f"Prediction Precision: {prec:.4f}")

In [ ]:

%matplotlib inline

list_tumor = [18, 19, 21, 22, 30, 36, 53, 64, 72, 84, 88, 90, 95, 97, 99, 116, 121, 122, 124, 130, 133, 142, 150, 
             154, 157, 158, 165, 167, 169, 172, 176, 183, 184, 191, 192, 194, 195, 201, 206, 209, 211, 214, 216, 
             221, 231, 237, 238, 239, 242, 247, 249, 259, 261, 266, 267, 283, 289, 290, 297, 300, 301, 309, 310, 
             312, 316, 320, 324, 325, 327, 336, 347, 351, 356, 373, 382, 388, 391, 392, 395, 397, 399, 401, 402, 
             414, 417, 421, 430, 433, 445, 446, 452, 455, 477, 481, 495, 498, 512, 538, 563, 565, 568, 569, 587, 
             588, 589, 591, 596, 601, 605, 616, 641, 649, 651, 654, 663, 668, 682, 683, 685, 686, 687, 706, 724, 
             727, 730, 733, 742, 747, 751, 756, 759, 764, 767, 774, 780, 792, 796, 797, 800, 803, 804, 810, 814, 
             818, 830, 1004, 1009]

tumor_y = []

test_origin = "m" #n

if test_origin == "m":
    for elem in ids:
        val = cc_test_pred.loc[cc_test_pred["BraTS21ID"]==elem]["MGMT_real_value"]
        if val.values[0] == 1 or int(elem[:5]) in list_tumor:
            tumor_y.append(1)
        else:
            tumor_y.append(0)
else:
    for elem in ids:
        tumor_y.append(1)

#print(cc_test_pred)
print(len(X))
print("Ids:")
print(list(ids))
print("Predictions:")
print(y_pred)
print("Real:")
print(y)
print("Probabilities:")
print(y_prob)
print("Tumors:")
print(tumor_y)

sel_idx = 19 #54,27,17,2
pred = y_pred[sel_idx]
real = y[sel_idx]
prob = y_prob[sel_idx]
tumor = tumor_y[sel_idx]

#print(X[sel_idx].shape)
#img_test = X[sel_idx].unsqueeze(0)
#print(img_test.shape)
#print("Methylation pred value: "+str(pred))
#print("Methylation real value: "+str(real))
#print("Methylation prob value: "+str(prob))
#print("Tumor real value: "+str(tumor))

In [ ]:
"""
%matplotlib notebook

j = 0
fig = plt.figure()
frames = []
img = ig
while (j < len(img)):
    frames.append([plt.imshow(img[j], cmap=cm.Greys_r, animated=True)])
    j += 1

ani = animation.ArtistAnimation(fig, frames, interval=50, blit=True,
                                repeat_delay=1000)
plt.show()
#%matplotlib inline
"""

In [ ]:
from medcam import medcam

In [ ]:
model = medcam.inject(m_model, label=0, backend="gcam", output_dir="attention_maps", save_maps=True)

In [ ]:
test_pred, X, y, y_pred, y_prob = predict(model, device, m_modelfile, m_test_loader, size)

In [ ]:
path = "attention_maps/part1/attention_map_34_0_0.nii.gz"
y, _ = loader_fc_3d_nifti(path)
print(y.shape)
img = y
fig = "Attention map"
slice_num = 96
plt.imshow(img, cmap=cm.Greys_r)
plt.show()